**Process**

1. open text file

2. record the section header pattern number.number.number

3. if a section (A) is found within another section (B). A will be copied into B.

In [10]:
import os

def extract_file_name(file_path):
    base_name = os.path.basename(file_path)
    file_name, _ = os.path.splitext(base_name)
    return file_name

import chardet

def detect_encoding(filepath):
    with open(filepath, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        return encoding
    
import re
import pickle

def text_collect(filename):
    pattern = re.compile(r"^\d+\.\d+\.\d+")
    sections = []
    current_section = None
    collected_text = []

    with open(filename, 'r',encoding=detect_encoding(filename),errors='replace') as file:
        for line in file:
            line = re.sub(r'[^\x00-\x7f]', r' ', line)
            line=line.lower()
            line = re.sub(r'[",!?*\[\]]', '', line)
            line = re.sub(r';', ' ', line)
            line = re.sub(r'\\', '', line)
            match = pattern.match(line)
            if match:
                if current_section:
                    sections.append({
                        "Section": current_section,
                        "Text": ' '.join(collected_text),
                        "Zoning Location": extract_file_name(filename)
                    })
                current_section = match.group()
                collected_text = [line[len(current_section):].strip()]
            elif current_section:
                collected_text.append(line.strip())

        if current_section:
            sections.append({
                "Section": current_section,
                "Text": ' '.join(collected_text),
                "Zoning Location": filename
            })

    with open(f"grouped_text.pkl", 'ab') as pkl_file:
        pickle.dump(sections, pkl_file)

#clear the file
with open("grouped_text.pkl", 'wb') as pkl_file:
    pass  # This will clear the file

import os
import glob

def process_files(directory):
    # Use glob to find all .txt files in the directory and subdirectories
    txt_files = glob.glob(os.path.join(directory, '**', '*.txt'), recursive=True)
    #print(txt_files)
    for file_path in txt_files:
        text_collect(file_path)

# Directory containing the text files
directory = r'C:\Users\clint\Desktop\Trucks\Trucks\nzlud\municipal_codes_all\real\set1'

# Process all text files in the directory
process_files(directory)


In [ ]:
with open('grouped_text.pkl', 'rb') as file:
    grouped_text_data = pickle.load(file)

grouped_text_data


In [ ]:
# Extract the 'Zoning Location' values
zoning_locations = [entry['Zoning Location'] for entry in grouped_text_data]


In [ ]:

# Find the unique entries
unique_zoning_locations = set(zoning_locations)

unique_zoning_locations
text_collect(r"C:\Users\clint\Desktop\Trucks\Trucks\october 7\code\AK_Homer.txt")



In [ ]:

import re
import pickle

# Load the pickle file
with open('grouped_text.pkl', 'rb') as file:
    data = pickle.load(file)


In [ ]:

# Function to find and replace patterns
def replace_patterns(data):
    pattern = re.compile(r'\d+\.\d+\.\d+')
    
    for entry in data:
        text = entry.get('Text')
        matches = pattern.findall(text)
        
        for match in matches:
            collected_texts = []
            for section_entry in data:
                section = section_entry.get('Section')
                if match in section:
                    collected_texts.append(section_entry.get('Text'))
            replacement_text = ' '.join(collected_texts)
            text = text.replace(match, replacement_text)
        entry['Text'] = text

    with open('modified_grouped_text.pkl', 'ab') as file:
        pickle.dump(data, file)

#clear the file
with open("modified_grouped_text.pkl", 'wb') as pkl_file:
    pass  # This will clear the file
# Replace patterns in the data
replace_patterns(data)